In [5]:
import os

current_year = '1945'
home_path = os.path.expanduser('~')
journals_path = os.path.join(home_path, 'Documents/Harry_Howard_Journal')
input_file = os.path.join(journals_path, f'{current_year}/{current_year}-entries.json')
embeddings_file = os.path.join(journals_path, 'journal-entries-with-embeddings.csv')

print(os.path.exists(embeddings_file))

True


In [2]:
import os
import pandas as pd
import tiktoken

from openai.embeddings_utils import get_embedding
import openai
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [10]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [11]:
# load & inspect dataset
df = pd.read_json(input_file)
df = df[['date', 'content']]
df = df.dropna()
df['combined'] = "Date: " + df['date'].astype(str).str.strip() + "; Text: " + df['content'].str.strip()

df.head(3)


,date,content,combined
0,1945-01-02,1945 was ushered in here in the state of Utah ...,Date: 1945-01-02; Text: 1945 was ushered in he...
1,1945-01-03,Learned this A.M. from Harold Dent that the PB...,Date: 1945-01-03; Text: Learned this A.M. from...
2,1945-01-04,My heart worries me. I am sure it is causing t...,Date: 1945-01-04; Text: My heart worries me. I...


In [12]:
encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
start_count = len(df)

df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens]

removed = start_count - len(df)
print(f'removed {removed}')

removed 0


In [13]:
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
print(f'adding {len(df)} embeddings')
df.to_csv(embeddings_file, mode='a', header=False, index=False)    # append mode


adding 180 embeddings


In [8]:
input_file = os.path.join(journals_path, f'{current_year}/{current_year}-analysis.json')

count_df = pd.read_json(input_file)

print(f'Found {len(count_df)} values')

Found 485 values
